In [2]:
!df -h

!cd /
!du -h --max-depth=1 | sort -hr
!find . -name Trash
!rm -r ./.local/share/Trash/files
!rm -r ./opt/ml/.local/share/Trash
!rm -r ./

Filesystem      Size  Used Avail Use% Mounted on
overlay          48G   19G   27G  42% /
tmpfs            64M     0   64M   0% /dev
tmpfs            45G     0   45G   0% /sys/fs/cgroup
shm             1.0G   44K  1.0G   1% /dev/shm
/dev/xvda1       48G   19G   27G  42% /etc/hosts
tmpfs            45G   12K   45G   1% /proc/driver/nvidia
udev             45G     0   45G   0% /dev/nvidia0
tmpfs            45G     0   45G   0% /proc/acpi
tmpfs            45G     0   45G   0% /proc/scsi
tmpfs            45G     0   45G   0% /sys/firmware
427M	.
378M	./pretrained
22M	./.git
5.3M	./configs
5.0M	./mmdet
3.9M	./demo
3.0M	./save_dir
2.1M	./.ipynb_checkpoints
1.7M	./resources
1.4M	./tests
308K	./docs
200K	./tools
164K	./readme.assets
56K	./.github
48K	./.dev_scripts
36K	./mmdet.egg-info
28K	./requirements
24K	./docker
24K	./custom_configs
8.0K	./work_dirs
8.0K	./__pycache__
4.0K	./mmcv_custom
4.0K	./.empty
rm: cannot remove './.local/share/Trash/files': No such file or directory
rm: cannot remov

In [45]:
import os
from urllib import request

from mmcv import Config

from utils import modify_path, modify_num_classes

## Model Name / Config name / Checkpoint URL

In [46]:
model_name = \
"vfnet"

model_cfg = \
"vfnet_x101_64x4d_fpn_mdconv_c3-c5_mstrain_2x_coco"

model_url = \
"https://openmmlab.oss-cn-hangzhou.aliyuncs.com/mmdetection/v2.0/vfnet/vfnet_x101_64x4d_fpn_mdconv_c3-c5_mstrain_2x_coco/vfnet_x101_64x4d_fpn_mdconv_c3-c5_mstrain_2x_coco_20201027pth-b5f6da5e.pth"

## Set Config Setting Values.

In [47]:
lr_gamma = 0.5

batch_size = 4

multi_scale_factor = [0.5, 0.75, 1.0, 1.25, 1.5]

## Load Config

In [48]:
cfg = Config.fromfile(os.path.join("./configs", model_name, f"{model_cfg}.py"))

## Modify Config

In [49]:
# modify lr_gamma * default lr.
cfg.optimizer["lr"] *= lr_gamma

# modify evaluation image scales.
if multi_scale_factor:
    test_h, test_w = cfg.data.test.pipeline[1]["img_scale"]
    cfg.data.test.pipeline[1]["img_scale"] = [(int(test_h * scale_factor), int(test_w * scale_factor)) for scale_factor in multi_scale_factor]
    cfg.data.test.pipeline[1]["flip"] = True

# add pretrained model from url.
cfg.load_from = os.path.abspath(os.path.join("pretrained", f"{model_cfg}.pth"))

# add wandb log.
#cfg.log_config = dict(interval=50, hooks=[dict(type="TextLoggerHook"), dict(type="WandbLoggerHook", init_kwargs=dict(project="pstage-3-od", name=model_cfg))])

# pth model file would be saved for most recent 3 epoch.
cfg.checkpoint_config = dict(interval=1, max_keep_ckpts=3)

# validation processed for every epoch, and best bbox mAP 50 model would be saved.
cfg.evaluation = dict(interval=1, metric='bbox', save_best="bbox_mAP_50")

# modify data path, save path.
modify_path(cfg, data_path="../../input/data", save_path=os.path.join("./save_dir", model_cfg))

# modify num of class.
modify_num_classes(cfg, class_num=11)

# batch size and num of worker.
cfg.data.samples_per_gpu = batch_size
cfg.data.workers_per_gpu = 2

cfg.data.train = dict(
        type='CocoDataset',
        ann_file='../../input/data/train_all.json', # '../../input/data/train.json',
        img_prefix='../../input/data',
        pipeline=[
            dict(type='LoadImageFromFile'),
            dict(
                type='InstaBoost',
                action_candidate=('normal', 'horizontal', 'skip'),
                action_prob=(1, 0, 0),
                scale=(0.8, 1.2),
                dx=15,
                dy=15,
                theta=(-1, 1),
                color_prob=0.5,
                hflag=False,
                aug_ratio=0.5),
            dict(type='LoadAnnotations', with_bbox=True),
            #dict(type='MosaicV2'),
            dict(
                type='Resize',
                img_scale=[(1333, 480), (1333, 960)],
                multiscale_mode='range',
                keep_ratio=True),
            dict(type='RandomFlip', flip_ratio=0.5),
            dict(
                type='Normalize',
                mean=[123.675, 116.28, 103.53],
                std=[58.395, 57.12, 57.375],
                to_rgb=True),
            dict(type='Pad', size_divisor=32),
            dict(type='DefaultFormatBundle'),
            dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels'])
        ],
        classes=('UNKNOWN', 'General trash', 'Paper', 'Paper pack', 'Metal',
                 'Glass', 'Plastic', 'Styrofoam', 'Plastic bag', 'Battery',
                 'Clothing'))

cfg.test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=([(666,400), (999,600), (1333,800), (1666,1000), (1999,1200)]), # img_scale=(1333, 800), #
        flip=True, # flip=False, #
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='RandomFlip'),
            dict(
                type='Normalize',
                mean=[123.675, 116.28, 103.53],
                std=[58.395, 57.12, 57.375],
                to_rgb=True),
            dict(type='Pad', size_divisor=32),
            dict(type='DefaultFormatBundle'),
            dict(type='Collect', keys=['img'])
        ])
]

## Save Config

In [50]:
os.makedirs(os.path.join("custom_configs"), exist_ok=True)
cfg.dump(os.path.join("custom_configs", f"{model_cfg}.py"))

## Pretrained Checkpoint Save.

In [44]:
if not os.path.isdir("pretrained"):
    os.makedirs("pretrained")

ckpt_name = os.path.join("pretrained", f"{model_cfg}.pth")

request.urlretrieve(model_url, ckpt_name)

('pretrained/vfnet_x101_64x4d_fpn_mdconv_c3-c5_mstrain_2x_coco.pth',
 <http.client.HTTPMessage at 0x7f4bf20f52d0>)